In [88]:
import random
import requests
from bs4 import BeautifulSoup
import time
import re
import json

In [89]:
def search(q):
    q = q.replace('\n', '')
    print("START : " + q)
    q = re.sub("\（.+語版\）", "", " ".join(q.split()[:-3]))
    query = re.sub('(!|\u3000|/|\s|>|<|\.)+', "%20", q)
    url_search = 'https://eiga.com/search/' + query
    res_search = requests.get(url_search )
    res_search.encoding = res_search.apparent_encoding
    soup_search = BeautifulSoup(res_search.content, "lxml")
    result =  soup_search.find('section', attrs={"id": "rslt-movie"})
    if(result != None):
        path = result.find('li', attrs={"class": "col-s-3"}).find('a')["href"]
        url_review = 'https://eiga.com' + path + 'review/all/'
        return url_review
    else:
        print("**************************************************")
        print(q + " HAS NO RESULT")
        print("**************************************************")
        return "error"

In [90]:
def scrape_review(query):
    page_num = 1
    data = {
        "id": -1,
        "reviews":{
            "eigacom":[],
            "filmarks":[],
            "coco":[],
        }
    }
    url_review=search(query)
    
    if(url_review == "error"):
        return "error"
    
    while(1):
        res = requests.get(url_review + str(page_num))
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.content, "lxml")
        if soup.find('div', attrs={"class": "user-review"}) == None: # ページ数の上限を超えたら
            print('DONE : ' + query )
            break
            
        for r in soup.find_all('div', attrs={"class": "user-review"}):
            title = r.find('h2',attrs={"class": "review-title"}).find('a')
            main_text =  r.find('div',attrs={"class": "txt-block"})
            tgl_btn = main_text.find('div',attrs={"class": "toggle-btn"})
            if tgl_btn != None:
                tgl_btn.decompose()
            d = title.text + "\n" +  main_text.text.replace("\n", "")
            data["reviews"]["eigacom"].append(d)
        page_num += 1
        time.sleep(1)
    return data

In [91]:
input_file = '../2018_movie_clean'

In [ ]:
num = 1
for q in open(input_file, 'r', encoding='utf-8').readlines():
    print(num)
    output_file = './{0}.json'.format(num)
    with open(output_file, 'w') as f:
        data = scrape_review(q)
        if(data == "error"):
            num += 1
            continue
        data["id"] = num
        jsn =  json.dumps(data,ensure_ascii=False, indent=2) 
        f.write(jsn)   
    f.close()
    num += 1

1
START : ZEN FOR NOTHING〜何でもない禅〜	2018	1	2
DONE : ZEN FOR NOTHING〜何でもない禅〜	2018	1	2

2
START : 映画 中二病でも恋がしたい！ -Take On Me-	2018	1	2
DONE : 映画 中二病でも恋がしたい！ -Take On Me-	2018	1	2

3
START : アンダー・ザ・ウォーター（ スウェーデン・ デンマーク・ フィンランド）	2018	1	5
**************************************************
アンダー・ザ・ウォーター（ スウェーデン・ デンマーク・ フィンランド） HAS NO RESULT
**************************************************
4
START : 嘘八百	2018	1	5
DONE : 嘘八百	2018	1	5

5
START : キングスマン: ゴールデン・サークル	2018	1	5
